In [ ]:
import os
import chromadb

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    Settings
)

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.chroma import ChromaVectorStore


In [ ]:
PDF_PATH = r""
CHROMA_DIR = "./chroma_db"
COLLECTION_NAME = "pdf_rag"


In [ ]:
embed_model = OllamaEmbedding(
    model_name="nomic-embed-text"
)

llm = Ollama(
    model="llama3.2:3b",
    temperature=0,
    request_timeout=120,
    additional_kwargs={
        "num_ctx": 2048   
    }
)

Settings.embed_model = embed_model
Settings.llm = llm


In [ ]:
documents = SimpleDirectoryReader(
    input_files=[PDF_PATH]
).load_data()

print(f"Loaded {len(documents)} document pages")


In [ ]:
chroma_client = chromadb.Client(
    settings=chromadb.Settings(
        persist_directory=CHROMA_DIR
    )
)

collection = chroma_client.get_or_create_collection(COLLECTION_NAME)

vector_store = ChromaVectorStore(
    chroma_collection=collection
)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)


In [ ]:
if collection.count() == 0:
    print("🔹 Building index (first run)...")
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context
    )
    print("✅ Index built and persisted")
else:
    print("🔹 Loading existing index...")
    index = VectorStoreIndex.from_vector_store(
        vector_store,
        storage_context=storage_context
    )
    print("✅ Index loaded")


In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=1,          # VERY IMPORTANT
    response_mode="compact"      # VERY IMPORTANT
)


In [ ]:
response = query_engine.query(
    "Summarize this document in simple terms"
)

print(response)


In [ ]:
chat_engine = index.as_chat_engine(
    chat_mode="context",
    similarity_top_k=1,
    response_mode="compact"
)

chat_engine.chat("What is this document about?")
chat_engine.chat("Explain it in bullet points")


In [ ]:
chat_response = chat_engine.chat("Explain it in bullet points")
print(chat_response.response)


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
import os

OUTPUT_DIR = "./data/pdfs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

topics = [
    "AWS Architecture Best Practices",
    "Kubernetes Networking Explained",
    "Terraform Modules and State Management",
    "CICD Pipeline Design",
    "Cloud Security and IAM",
    "Monitoring with Prometheus and Grafana",
    "Docker Internals",
    "FinOps Cost Optimization",
    "MLOps Pipelines",
    "FastAPI Backend Design",
    "Event Driven Architecture",
    "Microservices Patterns",
    "System Design Basics",
    "High Availability Systems",
    "Cloud Migration Strategy",
    "API Gateway Design",
    "Load Balancing Techniques",
    "Distributed Databases",
    "Caching with Redis",
    "Observability in Cloud Systems"
]

for i, topic in enumerate(topics, start=1):
    file_path = os.path.join(OUTPUT_DIR, f"{i:02d}_{topic.replace(' ', '_')}.pdf")
    c = canvas.Canvas(file_path, pagesize=A4)

    text = c.beginText(40, 800)
    text.setFont("Helvetica", 11)

    text.textLine(topic)
    text.textLine("=" * len(topic))
    text.textLine("")

    for _ in range(40):
        text.textLine(
            f"This document explains {topic.lower()} in detail. "
            "It covers concepts, best practices, trade-offs, and real-world usage. "
            "This content is intended for system design and cloud engineering discussions."
        )

    c.drawText(text)
    c.showPage()
    c.save()

print("✅ 20 PDFs generated successfully")
